In [2]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps

In [3]:
def computeH(im1_pts, im2_pts):
    'Compute homography matrix such that im2_pts = H @ im1_pts'
    n = im1_pts.shape[0]

    A = []
    b = []

    for i in range(n):
        x, y = im1_pts[i]
        x_prime, y_prime = im2_pts[i]

        A.append([x, y, 1, 0, 0, 0, -x*x_prime, -y*x_prime])
        b.append(x_prime)

        A.append([0, 0, 0, x, y, 1, -x*y_prime, -y*y_prime])
        b.append(y_prime)

    A = np.array(A)
    b = np.array(b)

    h, _, _, _ = np.linalg.lstsq(A, b, rcond=None)

    H = np.array([
        [h[0], h[1], h[2]],
        [h[3], h[4], h[5]],
        [h[6], h[7], 1]
    ])
    print(f'System of equations: {A} @ h = {b}')
    print(f'Solution: h = {h}')
    print(f'Recovered H: {H}')
    return H
    

def select_correspondences(img1, img2, num_points=4):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

    ax1.imshow(img1)
    ax1.set_title(f'Image 1 -- Select {num_points} correspondences')
    ax1.axis('on')

    ax2.imshow(img2)
    ax2.set_title(f'Image 2 -- Select {num_points} correspondences')
    ax2.axis('on')

    plt.tight_layout()

    print(f"Click on {num_points} points in each image to compute homography")
    
    img1_pts = plt.ginput(num_points, timeout=-1)
    im1_pts = np.array(img1_pts)

    ax1.plot(im1_pts[:, 0], im1_pts[:, 1], 'ro', markersize=15, markeredgewidth=2)
    for i, pt in enumerate(im1_pts):
        ax1.text(pt[0], pt[1], f'{i+1}', fontsize=16, ha='center', va='center', color='red')

    plt.draw()
    print("Click on the corresponding points in the second image")

    img2_pts = plt.ginput(num_points, timeout=-1)
    im2_pts = np.array(img2_pts)

    ax2.plot(im2_pts[:, 0], im2_pts[:, 1], 'ro', markersize=15, markeredgewidth=2)
    for i, pt in enumerate(im2_pts):
        ax2.text(pt[0], pt[1], f'{i+1}', fontsize=16, ha='center', va='center', color='red')
    
    plt.draw()
    plt.show()

    return im1_pts, im2_pts

def visualize_correspondences(img1, img2, im1_pts, im2_pts):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    
    ax1.imshow(img1)
    ax1.set_title('Image 1')
    ax1.plot(im1_pts[:, 0], im1_pts[:, 1], 'r+', markersize=15, markeredgewidth=2)
    for i, pt in enumerate(im1_pts):
        ax1.text(pt[0], pt[1], str(i+1), color='yellow', fontsize=12,
                fontweight='bold', ha='center', va='bottom')
    
    ax2.imshow(img2)
    ax2.set_title('Image 2')
    ax2.plot(im2_pts[:, 0], im2_pts[:, 1], 'r+', markersize=15, markeredgewidth=2)
    for i, pt in enumerate(im2_pts):
        ax2.text(pt[0], pt[1], str(i+1), color='yellow', fontsize=12,
                fontweight='bold', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

img1_path = '../images/set1/1.JPG'
img2_path = '../images/set1/2.JPG'

img1_pil = Image.open(img1_path)
img2_pil = Image.open(img2_path)
img1_pil = ImageOps.exif_transpose(img1_pil)  
img2_pil = ImageOps.exif_transpose(img2_pil)

max_size = 1000
img1_pil.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)
img2_pil.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)

img1 = np.array(img1_pil)
img2 = np.array(img2_pil)

im1_pts, im2_pts = select_correspondences(img1, img2, num_points=8)
visualize_correspondences(img1, img2, im1_pts, im2_pts)

H = computeH(im1_pts, im2_pts)

n = im1_pts.shape[0]
im1_pts_homogeneous = np.column_stack([im1_pts, np.ones(n)])
im2_pts_predicted = (H @ im1_pts_homogeneous.T).T
im2_pts_predicted = im2_pts_predicted[:, :2] / im2_pts_predicted[:, 2:3]

print("VERIFICATION:")
for i in range(n):
    error = np.linalg.norm(im2_pts[i] - im2_pts_predicted[i])
    print(f"Point {i+1}: Error = {error:.2f} pixels")

avg_error = np.mean([np.linalg.norm(im2_pts[i] - im2_pts_predicted[i]) for i in range(n)])
print(f"\nAverage error: {avg_error:.2f} pixels")

n = im1_pts.shape[0]
A = []
for i in range(n):
    x, y = im1_pts[i]
    x_prime, y_prime = im2_pts[i]
    A.append([-x, -y, -1, 0, 0, 0, x*x_prime, y*x_prime, x_prime])
    A.append([0, 0, 0, -x, -y, -1, x*y_prime, y*y_prime, y_prime])
A = np.array(A)



Click on 8 points in each image to compute homography
Click on the corresponding points in the second image
System of equations: [[ 4.15753462e+02  3.85809569e+02  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -9.41733234e+04 -8.73906598e+04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.15753462e+02
   3.85809569e+02  1.00000000e+00 -2.56882247e+05 -2.38380767e+05]
 [ 9.70405676e+01  2.09000422e+02  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -3.04256797e+04 -6.55290881e+04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.70405676e+01
   2.09000422e+02  1.00000000e+00 -4.66882307e+04 -1.00554440e+05]
 [ 4.27966667e+01  2.96023674e+02  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -2.48435968e+04 -1.71842655e+05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.27966667e+01
   2.96023674e+02  1.00000000e+00 -5.39756414e+03 -3.73348416e+04]
 [ 5.24659169e+01  2.96023674e+02  1.00000000e+00  0.00000000e+00
   0.00

In [4]:
def get_output_bounds(im_shape, H):
    h, w = im_shape[:2]
    corners = np.array([
        [0, 0, 1],
        [w - 1, 0, 1],
        [0, h - 1, 1],
        [w - 1, h - 1, 1]
    ]).T

    warped_corners = H @ corners
    warped_corners = warped_corners[:2, :] / warped_corners[2, :]

    x_min, x_max = warped_corners[0, :].min(), warped_corners[0, :].max()
    y_min, y_max = warped_corners[1, :].min(), warped_corners[1, :].max()

    return x_min, x_max, y_min, y_max

def warp_image_nearest_neighbor(im, H, fixed_output_shape=None):
    h_in, w_in, channels = im.shape

    if fixed_output_shape is not None:
        w_out, h_out = fixed_output_shape
        x_min, y_min = 0, 0
    else:
        x_min, x_max, y_min, y_max = get_output_bounds(im.shape, H)
        w_out = int(np.ceil(x_max - x_min))
        h_out = int(np.ceil(y_max - y_min))

    warped_im = np.zeros((h_out, w_out, channels), dtype=im.dtype)
    alpha_mask = np.zeros((h_out, w_out), dtype=np.float32)

    H_inv = np.linalg.inv(H)

    x_out, y_out = np.meshgrid(np.arange(w_out), np.arange(h_out))
    x_out = x_out + x_min
    y_out = y_out + y_min

    output_coords = np.stack([x_out.ravel(), y_out.ravel(), np.ones(h_out * w_out)])

    coords_in = H_inv @ output_coords
    coords_in = coords_in[:2, :] / coords_in[2, :]

    x_in = coords_in[0, :].reshape(h_out, w_out)
    y_in = coords_in[1, :].reshape(h_out, w_out)

    x_in_nearest_neighbor = np.round(x_in).astype(int)
    y_in_nearest_neighbor = np.round(y_in).astype(int)

    valid = (x_in_nearest_neighbor >= 0) & (x_in_nearest_neighbor < w_in) & (y_in_nearest_neighbor >= 0) & (y_in_nearest_neighbor < h_in)

    y_valid, x_valid = np.where(valid)

    warped_im[y_valid, x_valid, :] = im[y_in_nearest_neighbor[valid], x_in_nearest_neighbor[valid], :]
    alpha_mask[valid] = True

    return warped_im, alpha_mask

def warp_image_bilinear(im, H, fixed_output_shape=None):
    h_in, w_in, channels = im.shape

    if fixed_output_shape is not None:
        w_out, h_out = fixed_output_shape
        x_min, y_min = 0, 0
    else:
        x_min, x_max, y_min, y_max = get_output_bounds(im.shape, H)
        w_out = int(np.ceil(x_max - x_min))
        h_out = int(np.ceil(y_max - y_min))

    warped_im = np.zeros((h_out, w_out, channels), dtype=im.dtype)
    alpha_mask = np.zeros((h_out, w_out), dtype=np.float32)
    
    H_inv = np.linalg.inv(H)

    x_out, y_out = np.meshgrid(np.arange(w_out), np.arange(h_out))
    x_out = x_out + x_min
    y_out = y_out + y_min

    coords_out = np.stack([x_out.ravel(), y_out.ravel(), np.ones(h_out * w_out)])

    coords_in = H_inv @ coords_out
    coords_in = coords_in[:2, :] / coords_in[2, :]

    x_in = coords_in[0, :].reshape(h_out, w_out)
    y_in = coords_in[1, :].reshape(h_out, w_out)

    x0 = np.floor(x_in).astype(int)
    y0 = np.floor(y_in).astype(int)
    x1 = x0 + 1
    y1 = y0 + 1

    wx = x_in - x0
    wy = y_in - y0

    valid = (x0 >= 0) & (x1 < w_in) & (y0 >= 0) & (y1 < h_in)

    y_valid, x_valid = np.where(valid)

    for i in range(len(y_valid)):
        yv, xv = y_valid[i], x_valid[i]

        x0_i, x1_i = x0[yv, xv], x1[yv, xv]
        y0_i, y1_i = y0[yv, xv], y1[yv, xv]
        wx_i, wy_i = wx[yv, xv], wy[yv, xv]

        top_left = im[y0_i, x0_i, :] * (1 - wx_i) * (1 - wy_i)
        top_right = im[y0_i, x1_i, :] * wx_i * (1 - wy_i)
        bottom_left = im[y1_i, x0_i, :] * (1 - wx_i) * wy_i
        bottom_right = im[y1_i, x1_i, :] * wx_i * wy_i

        warped_im[yv, xv, :] = top_left + top_right + bottom_left + bottom_right
    
    alpha_mask[valid] = True

    return warped_im, alpha_mask
    


In [5]:
def rectify_image(img, corners_distorted, output_shape=(900, 600)):
    w_out, h_out = output_shape

    corners_rect = np.array([
        [0, 0],
        [w_out - 1, 0],
        [0, h_out - 1],
        [w_out - 1, h_out - 1]
    ], dtype=np.float64)

    H_rect = computeH(corners_distorted, corners_rect)

    rectified_nn, mask_nn = warp_image_nearest_neighbor(img, H_rect, output_shape)
    rectified_bil, mask_bil = warp_image_bilinear(img, H_rect, output_shape)

    return rectified_nn, rectified_bil, H

def visualize_rectification(img, corners, rectified_nn, rectified_bil, title="Rectification"):
    fig, axes = plt.subplots(1, 3, figsize=(18,6))

    axes[0].imshow(img)
    axes[0].plot(corners[[0,1,3,2,0], 0], corners[[0,1,3,2,0], 1], 'r-', linewidth=2)
    axes[0].plot(corners[:, 0], corners[:, 1], 'yo', markersize=10)
    for i, pt in enumerate(corners):
        axes[0].text(pt[0], pt[1]-20, str(i+1), color='yellow', fontsize=14, 
                    fontweight='bold', bbox=dict(boxstyle='round', facecolor='red', alpha=0.7))
    axes[0].set_title('Original')
    axes[0].axis('off')
    
    axes[1].imshow(rectified_nn)
    axes[1].set_title('Rectified - Nearest Neighbor')
    axes[1].axis('off')
    
    axes[2].imshow(rectified_bil)
    axes[2].set_title('Rectified - Bilinear')
    axes[2].axis('off')
    
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

img_pil = Image.open('../images/misc/flag.JPG')
img_pil = ImageOps.exif_transpose(img_pil)
img_rect = np.array(img_pil)

fig, ax = plt.subplots(figsize=(12, 8))
ax.imshow(img_rect)
ax.set_title('Select 4 corners of the rectangle')
plt.show()

corners = np.array(plt.ginput(4, timeout=0))

rect_nn, rect_bil, H_rect = rectify_image(img_rect, corners)

visualize_rectification(img_rect, corners, rect_nn, rect_bil, title="Rectification")

    

System of equations: [[ 6.04545455e+00  3.17404545e+03  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.04545455e+00
   3.17404545e+03  1.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [ 3.92227273e+02  2.06131818e+03  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -3.52612318e+05 -1.85312505e+06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.92227273e+02
   2.06131818e+03  1.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [ 2.48677273e+03  1.98931818e+03  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.48677273e+03
   1.98931818e+03  1.00000000e+00 -1.48957686e+06 -1.19160159e+06]
 [ 2.99731818e+03  3.15440909e+03  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -2.69458905e+06 -2.83581377e+06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+0

/var/folders/nc/dkzh6vs93fq3m59dzmkbrq1h0000gn/T/ipykernel_83603/2288927363.py:40: RuntimeWarning: divide by zero encountered in matmul
  coords_in = H_inv @ output_coords
/var/folders/nc/dkzh6vs93fq3m59dzmkbrq1h0000gn/T/ipykernel_83603/2288927363.py:40: RuntimeWarning: overflow encountered in matmul
  coords_in = H_inv @ output_coords
/var/folders/nc/dkzh6vs93fq3m59dzmkbrq1h0000gn/T/ipykernel_83603/2288927363.py:40: RuntimeWarning: invalid value encountered in matmul
  coords_in = H_inv @ output_coords
/var/folders/nc/dkzh6vs93fq3m59dzmkbrq1h0000gn/T/ipykernel_83603/2288927363.py:80: RuntimeWarning: divide by zero encountered in matmul
  coords_in = H_inv @ coords_out
/var/folders/nc/dkzh6vs93fq3m59dzmkbrq1h0000gn/T/ipykernel_83603/2288927363.py:80: RuntimeWarning: overflow encountered in matmul
  coords_in = H_inv @ coords_out
/var/folders/nc/dkzh6vs93fq3m59dzmkbrq1h0000gn/T/ipykernel_83603/2288927363.py:80: RuntimeWarning: invalid value encountered in matmul
  coords_in = H_inv @ co

In [ ]:
def compute_mosaic_bounds(images, homographies, ref_idx):
    all_corners = []
    
    for i, img in enumerate(images):
        h, w = img.shape[:2]
        corners = np.array([
            [0, 0, 1],
            [w - 1, 0, 1],
            [0, h - 1, 1],
            [w - 1, h - 1, 1]
        ]).T
        
        if i == ref_idx:
            warped_corners = corners
        else:
            warped_corners = homographies[i] @ corners
            warped_corners = warped_corners[:2, :] / warped_corners[2, :]
        
        all_corners.append(warped_corners.T[:, :2])
    
    all_corners = np.vstack(all_corners)
    x_min, y_min = all_corners.min(axis=0)
    x_max, y_max = all_corners.max(axis=0)
    
    return x_min, x_max, y_min, y_max

def create_alpha_mask(shape, blend_width=100):
    """Create alpha mask with linear falloff from edges"""
    h, w = shape[:2]
    alpha = np.ones((h, w), dtype=np.float32)
    
    for y in range(h):
        for x in range(w):
            dist_to_edge = min(x, w - 1 - x, y, h - 1 - y)
            if dist_to_edge < blend_width:
                alpha[y, x] = dist_to_edge / blend_width
    
    return alpha


def create_mosaic(images, homographies, ref_idx, blend_width=100):
    x_min, x_max, y_min, y_max = compute_mosaic_bounds(images, homographies, ref_idx)
    
    w_out = int(np.ceil(x_max - x_min))
    h_out = int(np.ceil(y_max - y_min))
    
    mosaic_sum = np.zeros((h_out, w_out, 3), dtype=np.float64)
    weight_sum = np.zeros((h_out, w_out), dtype=np.float64)
    
    for i, img in enumerate(images):
        
        alpha_base = create_alpha_mask(img.shape, blend_width)
        
        T_offset = np.array([
            [1, 0, -x_min],
            [0, 1, -y_min],
            [0, 0, 1]
        ])
        
        if i == ref_idx:
            H_adjusted = T_offset
        else:
            H_adjusted = T_offset @ homographies[i]
        
        warped_img, warped_mask = warp_image_bilinear(img, H_adjusted, 
                                                       fixed_output_shape=(w_out, h_out))
        
        alpha_img = np.stack([alpha_base] * 3, axis=2).astype(img.dtype)
        warped_alpha, _ = warp_image_bilinear(alpha_img, H_adjusted,
                                               fixed_output_shape=(w_out, h_out))
        warped_alpha = warped_alpha[:, :, 0] * warped_mask
        
        mosaic_sum += warped_img.astype(np.float64) * warped_alpha[:, :, np.newaxis]
        weight_sum += warped_alpha
    
    weight_sum[weight_sum == 0] = 1
    mosaic = mosaic_sum / weight_sum[:, :, np.newaxis]
    mosaic = np.clip(mosaic, 0, 255).astype(np.uint8)
    
    return mosaic

def load_images_from_set(set_path, image_numbers):
    """Load multiple images from a set"""
    images = []
    for num in image_numbers:
        img_path = f'{set_path}/{num}.JPG'
        img_pil = Image.open(img_path)
        img_pil = ImageOps.exif_transpose(img_pil)
        
        max_size = 1000
        img_pil.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)
        
        img = np.array(img_pil)
        images.append(img)
    
    return images

set_path = '../images/set1'
image_numbers = np.arange(1, 6)
images = load_images_from_set(set_path, image_numbers)
ref_idx = 2 

homographies = []

for i in range(len(images)):
    if i == ref_idx:
        homographies.append(np.eye(3))
    else:
        pts_i, pts_ref = select_correspondences(images[i], images[ref_idx], num_points=8)
        H_i_to_ref = computeH(pts_i, pts_ref)
        homographies.append(H_i_to_ref)
        
        pts_i_hom = np.column_stack([pts_i, np.ones(len(pts_i))])
        pts_ref_pred = (H_i_to_ref @ pts_i_hom.T).T
        pts_ref_pred = pts_ref_pred[:, :2] / pts_ref_pred[:, 2:3]
        avg_error = np.mean(np.linalg.norm(pts_ref - pts_ref_pred, axis=1))


mosaic = create_mosaic(images, homographies, ref_idx, blend_width=100)

plt.figure(figsize=(24, 8))
plt.imshow(mosaic)
plt.title('Panorama Mosaic - Set 1', fontsize=18)
plt.axis('off')
plt.tight_layout()
plt.savefig('../website_data/mosaic_set1.png', dpi=150, bbox_inches='tight')
plt.show()

Loaded 5 images, using image 3 as reference

=== Select correspondences: Image 1 -> Image 3 ===
Click on 8 points in each image to compute homography
Click on the corresponding points in the second image
System of equations: [[ 4.63720077e+02  2.77231370e+02  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -9.41141909e+04 -5.62654225e+04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.63720077e+02
   2.77231370e+02  1.00000000e+00 -1.30760204e+05 -7.81739509e+04]
 [ 5.36546194e+02  2.97021075e+02  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -1.47969173e+05 -8.19127289e+04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  5.36546194e+02
   2.97021075e+02  1.00000000e+00 -1.68284724e+05 -9.31590054e+04]
 [ 5.50003194e+02  1.70366958e+02  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -1.58646373e+05 -4.91417147e+04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  5.50003194e+02
   1.70366958e+02  1.00000000e+00 -1.0545752

/var/folders/nc/dkzh6vs93fq3m59dzmkbrq1h0000gn/T/ipykernel_83603/2288927363.py:80: RuntimeWarning: divide by zero encountered in matmul
  coords_in = H_inv @ coords_out
/var/folders/nc/dkzh6vs93fq3m59dzmkbrq1h0000gn/T/ipykernel_83603/2288927363.py:80: RuntimeWarning: overflow encountered in matmul
  coords_in = H_inv @ coords_out
/var/folders/nc/dkzh6vs93fq3m59dzmkbrq1h0000gn/T/ipykernel_83603/2288927363.py:80: RuntimeWarning: invalid value encountered in matmul
  coords_in = H_inv @ coords_out


Processing image 2/5...
Processing image 3/5...
Processing image 4/5...
Processing image 5/5...


: 